In [41]:
pip install tensorflow scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [42]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [43]:
data_dir = '/Users/rajat74/Downloads/data/food-101-tiny'
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')

In [44]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for subdir, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png'):
                img_path = os.path.join(subdir, file)
                image = Image.open(img_path)
                image = image.resize((128, 128))  # Resize the images to a common size
                images.append(np.array(image))
                labels.append(os.path.basename(subdir))
    return np.array(images), np.array(labels)

In [45]:
train_images, train_labels = load_images_from_folder(train_dir)
valid_images, valid_labels = load_images_from_folder(valid_dir)
print(f"Loaded {len(train_images)} training images with {len(set(train_labels))} classes.")
print(f"Loaded {len(valid_images)} validation images with {len(set(valid_labels))} classes.")

Loaded 1500 training images with 10 classes.
Loaded 500 validation images with 10 classes.


In [46]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator instance for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load images from directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)


Found 1500 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
from PIL import Image

# Load and preprocess data
def load_images_from_folder(folder):
    images = []
    labels = []
    for subdir, _, files in os.walk(folder):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png'):
                img_path = os.path.join(subdir, file)
                image = Image.open(img_path)
                image = image.resize((128, 128))
                images.append(np.array(image))
                labels.append(os.path.basename(subdir))
    return np.array(images), np.array(labels)

data_dir = '/Users/rajat74/Downloads/data/food-101-tiny'
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')

train_images, train_labels = load_images_from_folder(train_dir)
valid_images, valid_labels = load_images_from_folder(valid_dir)

# Normalize images
train_images = train_images / 255.0
valid_images = valid_images / 255.0

# Encode labels
encoder = LabelEncoder()
train_labels_encoded = encoder.fit_transform(train_labels)
valid_labels_encoded = encoder.transform(valid_labels)

# Define the model creation function
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(len(np.unique(train_labels)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model with KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20]
}

# Setup the GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

# Fit the GridSearchCV
grid_result = grid.fit(train_images, train_labels_encoded)

# Retrieve the best parameters
best_params = grid_result.best_params_
print(f"Best params: {best_params}")

# Use the best parameters to train the final model
model = create_model()
model.fit(
    train_images, 
    tf.keras.utils.to_categorical(train_labels_encoded), 
    epochs=best_params['epochs'], 
    batch_size=best_params['batch_size'], 
    validation_data=(valid_images, tf.keras.utils.to_categorical(valid_labels_encoded))
)


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()


In [ ]:
# Save the model
model.save('food_classification_model.h5')

# Load the model
model = tf.keras.models.load_model('food_classification_model.h5')


In [ ]:
# Load a new image
new_image_path = '/Users/rajat74/Downloads/data/food-101-tiny/valid/ice_cream/1272987.jpg'
new_image = Image.open(new_image_path)
new_image = new_image.resize((128, 128))
new_image = np.array(new_image) / 255.0  # Normalize to [0, 1]
new_image = np.expand_dims(new_image, axis=0)  # Add batch dimension

# Make a prediction
predictions = model.predict(new_image)
predicted_class = np.argmax(predictions, axis=1)
print(f"Predicted class: {predicted_class}")
